In [ ]:
from google.colab import drive

MOUNTPOINT = '/content/gdrive'
drive.mount(MOUNTPOINT)

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

input_dataset = '/content/gdrive/My Drive/Colab Notebooks/datasets/ISIC2018_Task1-2_Training_Data/ISIC2018_Task1-2_Training_Input_x2'
output_dataset = '/content/gdrive/My Drive/Colab Notebooks/datasets/ISIC2018_Task1-2_Training_Data/ISIC2018_Task1_Training_GroundTruth_x2'

train_x = tf.keras.utils.image_dataset_from_directory(directory=input_dataset, labels=None, shuffle=False, image_size=(256,256), validation_split=0.1, subset='training')
val_x = tf.keras.utils.image_dataset_from_directory(directory=input_dataset, labels=None, shuffle=False, image_size=(256,256), validation_split=0.1, subset='validation')

# normalize
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(scale= 1./255)
train_x = train_x.map(lambda x: normalization_layer(x))


# 10% test and val split with 80% train

#train_x = train_x.skip(9)
test_x = train_x.take(9)
train_x = train_x.skip(9)

print(len(val_x))
print(len(train_x))
print(len(test_x))

train_y = tf.keras.utils.image_dataset_from_directory(directory=output_dataset, labels=None, shuffle=False, image_size=(256,256), validation_split=0.1, subset='training', color_mode='grayscale')
val_y = tf.keras.utils.image_dataset_from_directory(directory=output_dataset, labels=None, shuffle=False, image_size=(256,256), validation_split=0.1, subset='validation', color_mode='grayscale')
train_y = train_y.map(lambda x: normalization_layer(x))

# 10% test and val split with 80% train

test_y = train_y.take(9)
train_y = train_y.skip(9)

print(len(val_y))
print(len(train_y))
print(len(test_y))

# need to convert y data to tf.dataset and then zip them together
train_ds = tf.data.Dataset.zip((train_x , train_y))
val_ds = tf.data.Dataset.zip((val_x , val_y))
test_ds = tf.data.Dataset.zip((test_x , test_y))



In [ ]:
# copy paste contents of model.py into here

In [ ]:
height = 256
width = 256
channels = 3

unet = model(height, width, channels)

unet.summary()
history =  unet.fit(train_ds, validation_data=val_ds, batch_size=16, epochs=10)

predict_test = unet.predict(test_x)